<a href="https://colab.research.google.com/github/thwlswkdgh/GAN/blob/main/code_practice/CycleGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Generator

In [1]:
pip install git+https://www.github.com/keras-team/keras-contrib.git

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-bs29n5dk
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-bs29n5dk
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-py3-none-any.whl size=101077 sha256=f6f4b1f3a797907498def79c8a3aa2602a057d2d359a9c10b49dcb0a27848d81
  Stored in directory: /tmp/pip-ephem-wheel-cache-otp5l_lt/wheels/bb/1f/f2/b57495012683b6b20bbae94a3915ec79753111452d79886abc
Successfully built keras-contrib


In [2]:
#!/usr/bin/env python
import sys
import numpy as np
from keras.layers import Dense, Reshape, Input, BatchNormalization, Concatenate
from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization
from keras.layers.core import Activation
from keras.layers.convolutional import UpSampling2D, Convolution2D, MaxPooling2D,Deconvolution2D
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam, SGD, Nadam,Adamax
from keras import initializers
from keras.utils.vis_utils import plot_model

class Generator(object):
    def __init__(self, width = 28, height= 28, channels = 1):
        
        self.W = width
        self.H = height
        self.C = channels
        self.SHAPE = (width,height,channels)

        self.Generator = self.model()
        self.OPTIMIZER = Adam(lr=2e-4, beta_1=0.5,decay=1e-5)
        self.Generator.compile(loss='binary_crossentropy', optimizer=self.OPTIMIZER,metrics=['accuracy'])

        self.save_model()
        self.summary()

    def model(self):
        input_layer = Input(shape=self.SHAPE)
        
        down_1 = Convolution2D(64  , kernel_size=4, strides=2, padding='same',activation=LeakyReLU(alpha=0.2))(input_layer)
        norm_1 = InstanceNormalization()(down_1)

        down_2 = Convolution2D(64*2, kernel_size=4, strides=2, padding='same',activation=LeakyReLU(alpha=0.2))(norm_1)
        norm_2 = InstanceNormalization()(down_2)

        down_3 = Convolution2D(64*4, kernel_size=4, strides=2, padding='same',activation=LeakyReLU(alpha=0.2))(norm_2)
        norm_3 = InstanceNormalization()(down_3)

        down_4 = Convolution2D(64*8, kernel_size=4, strides=2, padding='same',activation=LeakyReLU(alpha=0.2))(norm_3)
        norm_4 = InstanceNormalization()(down_4)


        upsample_1 = UpSampling2D()(norm_4)
        up_conv_1 = Convolution2D(64*4, kernel_size=4, strides=1, padding='same',activation='relu')(upsample_1)
        norm_up_1 = InstanceNormalization()(up_conv_1)
        add_skip_1 = Concatenate()([norm_up_1,norm_3])

        upsample_2 = UpSampling2D()(add_skip_1)
        up_conv_2 = Convolution2D(64*2, kernel_size=4, strides=1, padding='same',activation='relu')(upsample_2)
        norm_up_2 = InstanceNormalization()(up_conv_2)
        add_skip_2 = Concatenate()([norm_up_2,norm_2])

        upsample_3 = UpSampling2D()(add_skip_2)
        up_conv_3 = Convolution2D(64, kernel_size=4, strides=1, padding='same',activation='relu')(upsample_3)
        norm_up_3 = InstanceNormalization()(up_conv_3)
        add_skip_3 = Concatenate()([norm_up_3,norm_1])

        last_upsample = UpSampling2D()(add_skip_3)
        
        output_layer = Convolution2D(3, kernel_size=4, strides=1, padding='same',activation='tanh')(last_upsample)
        return Model(input_layer,output_layer)

    def summary(self):
        return self.Generator.summary()

    def save_model(self):
        plot_model(self.Generator, to_file='/content/gdrive/My Drive/Colab Notebooks/cycleGAN/monet2photo_out/Generator_Model.png', show_shapes=True)



Discriminator

In [3]:
#!/usr/bin/env python3
import sys
import numpy as np
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, BatchNormalization, Lambda, concatenate
from keras.layers.core import Activation
from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization
from keras.layers.convolutional import Convolution2D
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam, SGD,Nadam, Adamax
import keras.backend as K
from keras.utils.vis_utils import plot_model


class Discriminator(object):
    def __init__(self, width = 28, height= 28, channels = 1):
        self.W = width
        self.H = height
        self.C = channels
        self.CAPACITY = width*height*channels
        self.SHAPE = (width,height,channels)
        
        self.Discriminator = self.model()
        self.OPTIMIZER = Adam(lr=2e-4, beta_1=0.5,decay=1e-5)
        self.Discriminator.compile(loss='mse', optimizer=self.OPTIMIZER, metrics=['accuracy'] )

        self.save_model()
        self.summary()

    def model(self):
        input_layer = Input(self.SHAPE)

        up_layer_1 = Convolution2D(64, kernel_size=4, strides=2, padding='same',activation=LeakyReLU(alpha=0.2))(input_layer)

        up_layer_2 = Convolution2D(64*2, kernel_size=4, strides=2, padding='same',activation=LeakyReLU(alpha=0.2))(up_layer_1)
        norm_layer_1 = InstanceNormalization()(up_layer_2)

        up_layer_3 = Convolution2D(64*4, kernel_size=4, strides=2, padding='same',activation=LeakyReLU(alpha=0.2))(norm_layer_1)
        norm_layer_2 = InstanceNormalization()(up_layer_3)

        up_layer_4 = Convolution2D(64*8, kernel_size=4, strides=2, padding='same',activation=LeakyReLU(alpha=0.2))(norm_layer_2)
        norm_layer_3 =InstanceNormalization()(up_layer_4)

        output_layer = Convolution2D(1, kernel_size=4, strides=1, padding='same')(norm_layer_3)
        output_layer_1 = Flatten()(output_layer)
        output_layer_2 = Dense(1, activation='sigmoid')(output_layer_1)
        
        return Model(input_layer,output_layer_2)

    def summary(self):
        return self.Discriminator.summary()

    def save_model(self):
        plot_model(self.Discriminator, to_file='/content/gdrive/My Drive/Colab Notebooks/cycleGAN/monet2photo_out/Discriminator_Model.png', show_shapes=True)



GAN

In [4]:
#!/usr/bin/env python3
import sys
import numpy as np
from keras.models import Sequential, Model
from keras.layers import Input
from tensorflow.keras.optimizers import Adam, SGD
from keras.utils.vis_utils import plot_model

class GAN(object):
    def __init__(self, model_inputs=[],model_outputs=[],lambda_cycle=10.0,lambda_id=1.0):
        self.OPTIMIZER = SGD(lr=2e-4,nesterov=True)

        self.inputs = model_inputs
        self.outputs = model_outputs
        self.gan_model = Model(self.inputs,self.outputs)
        self.OPTIMIZER = Adam(lr=2e-4, beta_1=0.5)
        self.gan_model.compile(loss=['mse', 'mse',
                                    'mae', 'mae',
                                    'mae', 'mae'],
                            loss_weights=[  1, 1,
                                            lambda_cycle, lambda_cycle,
                                            lambda_id, lambda_id ],
                            optimizer=self.OPTIMIZER)
        self.save_model()
        self.summary()

    def model(self):
        model = Model()
        return model

    def summary(self):
        return self.gan_model.summary()

    def save_model(self):
        plot_model(self.gan_model, to_file='/content/gdrive/My Drive/Colab Notebooks/cycleGAN/monet2photo_out/GAN_Model.png', show_shapes=True)


Trainer



In [5]:
#!/usr/bin/env python3
#from gan import GAN
#from generator import Generator
#from discriminator import Discriminator
from keras.layers import Input
from keras.datasets import mnist
from random import randint
import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy
import os
from PIL import Image
import numpy as np
from tensorflow.keras.models import load_model
from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization


class Trainer:
    def __init__(self, height = 64, width = 64, epochs = 50000, batch = 32, checkpoint = 50, train_data_path_A = '',train_data_path_B = '',test_data_path_A='',test_data_path_B='',lambda_cycle=10.0,lambda_id=1.0):
        self.EPOCHS = epochs
        self.BATCH = batch
        self.RESIZE_HEIGHT = height
        self.RESIZE_WIDTH = width
        self.CHECKPOINT = checkpoint
        
        self.X_train_A, self.H_A, self.W_A, self.C_A = self.load_data(train_data_path_A)
        self.X_train_B, self.H_B, self.W_B, self.C_B  = self.load_data(train_data_path_B)
        self.X_test_A, self.H_A_test, self.W_A_test, self.C_A_test = self.load_data(test_data_path_A)
        self.X_test_B, self.H_B_test, self.W_B_test, self.C_B_test  = self.load_data(test_data_path_B)

        self.generator_A_to_B = Generator(height=self.H_A, width=self.W_A, channels=self.C_A)
        self.generator_B_to_A = Generator(height=self.H_B, width=self.W_B, channels=self.C_B)
        #if(self.EPOCHS ==0 and self.BATCH == 1):
        print ('loading')
        self.generator_A_to_B.Generator = load_model('/content/gdrive/My Drive/Colab Notebooks/cycleGAN/monet2photo_out/generator_A_to_B.h5', custom_objects={'InstanceNormalization':InstanceNormalization})
        self.generator_B_to_A.Generator = load_model('/content/gdrive/My Drive/Colab Notebooks/cycleGAN/monet2photo_out/generator_B_to_A.h5', custom_objects={'InstanceNormalization':InstanceNormalization})

        self.orig_A = Input(shape=(self.W_A, self.H_A, self.C_A))
        self.orig_B = Input(shape=(self.W_B, self.H_B, self.C_B))

        self.fake_B = self.generator_A_to_B.Generator(self.orig_A)
        self.fake_A = self.generator_B_to_A.Generator(self.orig_B)
        self.reconstructed_A = self.generator_B_to_A.Generator(self.fake_B)
        self.reconstructed_B = self.generator_A_to_B.Generator(self.fake_A)
        self.id_A = self.generator_B_to_A.Generator(self.orig_A)
        self.id_B = self.generator_A_to_B.Generator(self.orig_B)
        
        

        self.discriminator_A = Discriminator(height=self.H_A, width=self.W_A, channels=self.C_A)
        self.discriminator_B = Discriminator(height=self.H_B, width=self.W_B, channels=self.C_B)
        #if(self.EPOCHS ==0 and self.BATCH == 1):
        self.discriminator_A.Discriminator = load_model('/content/gdrive/My Drive/Colab Notebooks/cycleGAN/monet2photo_out/discriminator_A.h5', custom_objects={'InstanceNormalization':InstanceNormalization})
        self.discriminator_B.Discriminator = load_model('/content/gdrive/My Drive/Colab Notebooks/cycleGAN/monet2photo_out/discriminator_B.h5', custom_objects={'InstanceNormalization':InstanceNormalization})

        self.discriminator_A.trainable = False
        self.discriminator_B.trainable = False
        self.valid_A = self.discriminator_A.Discriminator(self.fake_A)
        self.valid_B = self.discriminator_B.Discriminator(self.fake_B)

        
        model_inputs  = [self.orig_A,self.orig_B]
        model_outputs = [self.valid_A, self.valid_B,self.reconstructed_A,self.reconstructed_B,self.id_A, self.id_B]
        self.gan = GAN(model_inputs=model_inputs,model_outputs=model_outputs,lambda_cycle=lambda_cycle,lambda_id=lambda_id)
        #if(self.EPOCHS ==0 and self.BATCH == 1):
        self.gan.GAN = load_model('/content/gdrive/My Drive/Colab Notebooks/cycleGAN/monet2photo_out/gan_model.h5', custom_objects={'InstanceNormalization':InstanceNormalization})

    def load_data(self,data_path,amount_of_data = 1.0):
        listOFFiles = self.grabListOfFiles(data_path,extension="jpg")
        X_train = np.array(self.grabArrayOfImages(listOFFiles))
        height, width, channels = np.shape(X_train[0])
        X_train = X_train[:int(amount_of_data*float(len(X_train)))]
        X_train = (X_train.astype(np.float32) - 127.5)/127.5
        X_train = np.expand_dims(X_train, axis=3)
        return X_train, height, width, channels

    def grabListOfFiles(self,startingDirectory,extension=".webp"):
        listOfFiles = []
        for file in os.listdir(startingDirectory):
            if file.endswith(extension):
                listOfFiles.append(os.path.join(startingDirectory, file))
        return listOfFiles

    def grabArrayOfImages(self,listOfFiles,gray=False):
        imageArr = []
        for f in listOfFiles:
            if gray:
                im = Image.open(f).convert("L")
            else:
                im = Image.open(f).convert("RGB")
            im = im.resize((self.RESIZE_WIDTH,self.RESIZE_HEIGHT))
            imData = np.asarray(im)
            imageArr.append(imData)
        return imageArr


    def train(self):
        for e in range(self.EPOCHS):
            b = 0
            X_train_A_temp = deepcopy(self.X_train_A)
            X_train_B_temp = deepcopy(self.X_train_B)
        
            while min(len(X_train_A_temp),len(X_train_B_temp))>self.BATCH:
                # Keep track of Batches
                b=b+1

                # Train Discriminator
                # Grab Real Images for this training batch

                count_real_images = int(self.BATCH)
                starting_indexs = randint(0, (min(len(X_train_A_temp),len(X_train_B_temp))-count_real_images))
                real_images_raw_A = X_train_A_temp[ starting_indexs : (starting_indexs + count_real_images) ]
                real_images_raw_B = X_train_B_temp[ starting_indexs : (starting_indexs + count_real_images) ]

                # Delete the images used until we have none left
                X_train_A_temp = np.delete(X_train_A_temp,range(starting_indexs,(starting_indexs + count_real_images)),0)
                X_train_B_temp = np.delete(X_train_B_temp,range(starting_indexs,(starting_indexs + count_real_images)),0)
                batch_A = real_images_raw_A.reshape( count_real_images, self.W_A, self.H_A, self.C_A )
                batch_B = real_images_raw_B.reshape( count_real_images, self.W_B, self.H_B, self.C_B )

                self.discriminator_A.Discriminator.trainable = True
                self.discriminator_B.Discriminator.trainable = True
                x_batch_A = batch_A
                x_batch_B = batch_B
                y_batch_A = np.ones([count_real_images,1])
                y_batch_B = np.ones([count_real_images,1])
                # Now, train the discriminator with this batch of reals
                discriminator_loss_A_real = self.discriminator_A.Discriminator.train_on_batch(x_batch_A,y_batch_A)[0]
                discriminator_loss_B_real = self.discriminator_B.Discriminator.train_on_batch(x_batch_B,y_batch_B)[0]

                x_batch_B = self.generator_A_to_B.Generator.predict(batch_A)
                x_batch_A = self.generator_B_to_A.Generator.predict(batch_B)
                y_batch_A = np.zeros([self.BATCH,1])
                y_batch_B = np.zeros([self.BATCH,1])
                # Now, train the discriminator with this batch of fakes
                discriminator_loss_A_fake = self.discriminator_A.Discriminator.train_on_batch(x_batch_A,y_batch_A)[0]
                discriminator_loss_B_fake = self.discriminator_B.Discriminator.train_on_batch(x_batch_B,y_batch_B)[0]    

                self.discriminator_A.Discriminator.trainable = False
                self.discriminator_B.Discriminator.trainable = False

                discriminator_loss_A = 0.5*(discriminator_loss_A_real+discriminator_loss_A_fake)
            
                # In practice, flipping the label when training the generator improves convergence
                if self.flipCoin(chance=0.9):
                    y_generated_labels = np.ones([self.BATCH,1])
                else:
                    y_generated_labels = np.zeros([self.BATCH,1])
                generator_loss = self.gan.gan_model.train_on_batch([x_batch_A, x_batch_B],
                                                        [y_generated_labels, y_generated_labels,
                                                        x_batch_A, x_batch_B,
                                                        x_batch_A, x_batch_B])    

                print ('Epoch: '+str(int(e))+' Batch: '+str(int(b))+', [Discriminator_A :: Loss: '+str(discriminator_loss_A)+'], [ Generator :: Loss: '+str(generator_loss)+']')
                if b % self.CHECKPOINT == 0 :
                    label = str(e)+'_'+str(b)
                    self.plot_checkpoint(label)

            print ('Epoch: '+str(int(e))+', [Discriminator_A :: Loss: '+str(discriminator_loss_A)+'], [ Generator :: Loss: '+str(generator_loss)+']')
                        
            #if e % self.CHECKPOINT == 0 :
                #self.plot_checkpoint(e)
        return

    def flipCoin(self,chance=0.5):
        return np.random.binomial(1, chance)

    def plot_checkpoint(self,b):
        orig_filename = "/content/gdrive/My Drive/Colab Notebooks/cycleGAN/monet2photo_out/batch/batch_check_"+str(b)+"_original.png"

        image_A = self.X_test_A[6]
        image_A = np.reshape(image_A, [self.W_A_test,self.H_A_test,self.C_A_test])
        print("Image_A shape: " +str(np.shape(image_A)))
        fake_B = self.generator_A_to_B.Generator.predict(image_A.reshape(1, self.W_A, self.H_A, self.C_A ))
        fake_B = np.reshape(fake_B, [self.W_A_test,self.H_A_test,self.C_A_test])
        print("fake_B shape: " +str(np.shape(fake_B)))
        reconstructed_A = self.generator_B_to_A.Generator.predict(fake_B.reshape(1, self.W_A, self.H_A, self.C_A ))
        reconstructed_A = np.reshape(reconstructed_A, [self.W_A_test,self.H_A_test,self.C_A_test])
        print("reconstructed_A shape: " +str(np.shape(reconstructed_A)))
        # from IPython import embed; embed()

        checkpoint_images = np.array([image_A, fake_B, reconstructed_A])

        # Rescale images 0 - 1
        checkpoint_images = 0.5 * checkpoint_images + 0.5

        titles = ['Original', 'Translated', 'Reconstructed']
        fig, axes = plt.subplots(1, 3)
        for i in range(3):
            image = checkpoint_images[i]
            image = np.reshape(image, [self.H_A_test,self.W_A_test,self.C_A_test])
            axes[i].imshow(image)
            axes[i].set_title(titles[i])
            axes[i].axis('off')
        fig.savefig("/content/gdrive/My Drive/Colab Notebooks/cycleGAN/monet2photo_out/batch/batch_check_"+str(b)+".png")
        plt.close('all')

        self.gan.gan_model.save('/content/gdrive/My Drive/Colab Notebooks/cycleGAN/monet2photo_out/gan_model')
        self.generator_A_to_B.Generator.save('/content/gdrive/My Drive/Colab Notebooks/cycleGAN/monet2photo_out/generator_A_to_B')
        self.generator_B_to_A.Generator.save('/content/gdrive/My Drive/Colab Notebooks/cycleGAN/monet2photo_out/generator_B_to_A')
        self.discriminator_A.Discriminator.save('/content/gdrive/My Drive/Colab Notebooks/cycleGAN/monet2photo_out/discriminator_A')
        self.discriminator_B.Discriminator.save('/content/gdrive/My Drive/Colab Notebooks/cycleGAN/monet2photo_out/discriminator_B')


        return


In [ ]:
#!/usr/bin/env python3
#from train import Trainer
from google.colab import drive
drive.mount('/content/gdrive')

# Command Line Argument Method
HEIGHT  = 64
WIDTH   = 64
CHANNEL = 3
EPOCHS = 60
BATCH = 1
CHECKPOINT = 100
TRAIN_PATH_A = "/content/gdrive/My Drive/Colab Notebooks/cycleGAN/monet2photo/trainA/"
TRAIN_PATH_B = "/content/gdrive/My Drive/Colab Notebooks/cycleGAN/monet2photo/trainB/"
TEST_PATH_A = "/content/gdrive/My Drive/Colab Notebooks/cycleGAN/monet2photo/testA/"
TEST_PATH_B = "/content/gdrive/My Drive/Colab Notebooks/cycleGAN/monet2photo/testB/"

trainer = Trainer(height=HEIGHT,width=WIDTH,epochs =EPOCHS,\
                 batch=BATCH,\
                 checkpoint=CHECKPOINT,\
                 train_data_path_A=TRAIN_PATH_A,\
                 train_data_path_B=TRAIN_PATH_B,\
                 test_data_path_A=TEST_PATH_A,\
                 test_data_path_B=TEST_PATH_B,\
                 lambda_cycle=10.0,\
                 lambda_id=1.0)
trainer.train()



Mounted at /content/gdrive


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 64, 64, 3)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 32, 32, 64)   3136        ['input_1[0][0]']                
                                                                                                  
 instance_normalization (Instan  (None, 32, 32, 64)  2           ['conv2d[0][0]']                 
 ceNormalization)                                                                                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 16, 16, 128)  131200      ['instance_normalization[0][0

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 64, 64, 3)]  0           []                               
                                                                                                  
 input_3 (InputLayer)           [(None, 64, 64, 3)]  0           []                               
                                                                                                  
 model_1 (Functional)           (None, 64, 64, 3)    6171025     ['input_4[0][0]',                
                                                                  'model[0][0]',                  
                                                                  'input_3[0][0]']                
                                                                                            

...

INFO:tensorflow:Assets written to: /content/gdrive/My Drive/Colab Notebooks/cycleGAN/monet2photo_out/discriminator_A/assets


INFO:tensorflow:Assets written to: /content/gdrive/My Drive/Colab Notebooks/cycleGAN/monet2photo_out/discriminator_A/assets


INFO:tensorflow:Assets written to: /content/gdrive/My Drive/Colab Notebooks/cycleGAN/monet2photo_out/discriminator_B/assets


INFO:tensorflow:Assets written to: /content/gdrive/My Drive/Colab Notebooks/cycleGAN/monet2photo_out/discriminator_B/assets


Epoch: 2 Batch: 501, [Discriminator_A :: Loss: 0.08618234726600349], [ Generator :: Loss: [0.7915129065513611, 0.0020406097173690796, 0.0, 0.03703147917985916, 0.031533919274806976, 0.05445541813969612, 0.049362875521183014]]
Epoch: 2 Batch: 502, [Discriminator_A :: Loss: 0.010445314459502697], [ Generator :: Loss: [1.032920479774475, 0.0016321911243721843, 0.0, 0.04024844989180565, 0.049591872841119766, 0.08286384493112564, 0.050021156668663025]]
Epoch: 2 Batch: 503, [Discriminator_A :: Loss: 0.12233020402345574], [ Generator :: Loss: [2.8070921897888184, 0.9582189917564392, 1.0, 0.03432123735547066, 0.03948938846588135, 0.0518869087100029, 0.05888006463646889]]
Epoch: 2 Batch: 504, [Discriminator_A :: Loss: 0.0001913590676849708], [ Generator :: Loss: [1.0800431966781616, 0.00040038078441284597, 0.0, 0.054910577833652496, 0.03552551567554474, 0.11452438682317734, 0.060757532715797424]]
Epoch: 2 Batch: 505, [Discriminator_A :: Loss: 0.0015582264750264585], [ Generator :: Loss: [0.8037